# Dressmaker - Samples

The data model of the tables.

![](../src/img/dressmaker_str.svg)

In [1]:
import findspark
import pandas as pd
findspark.init()

SVR = '192.168.31.31'
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
from pyspark.sql import Window

sc = (SparkSession.builder.appName('app17-0') 
      .master(f'spark://{SVR}:7077') 
      .config('spark.sql.warehouse.dir', f'hdfs://{SVR}:9000/user/hive/warehouse') 
      .config('spark.cores.max', '4') 
      .config('spark.executor.instances', '1') 
      .config('spark.executor.cores', '2') 
      .config('spark.executor.memory', '10g') 
      .enableHiveSupport().getOrCreate())

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


In [2]:
jmcust = sc.read.table('sqlzoo.jmcust')
dressmaker = sc.read.table('sqlzoo.dressmaker')
dress_order = sc.read.table('sqlzoo.dress_order')
construction = sc.read.table('sqlzoo.construction')
quantities = sc.read.table('sqlzoo.quantities')
order_line = sc.read.table('sqlzoo.order_line')
garment = sc.read.table('sqlzoo.garment')
material = sc.read.table('sqlzoo.material')

## 1.
The "central" table in this database is order_line - every garment ordered takes one line in this table. Many of the fields in this table are references to other tables. The fields of this table have the following meaning: 

- order_ref

This is a link to the dress_order table. We can join the dress_order table to find information such as the the date of the order and the customer number for a given garment order.

- line_no

The line number is used to distinguish different items on the same order - for example order number 5 has three lines - 1, 2 and 3.

- ol_style

Indicates the article of clothing ordered. For example ol_style 1 indicates trousers - we can see this by joining to the garments table. Line 1 in the garment table is trousers.

- ol_size

The size of the item ordered is given here - this is particularly important when it comes to working out how much material is required to build the item. We can see from the quantities table that trousers (style 1) in size 8 takes 2.7 meters - whereas trousers in size 12 needs 2.8 meters.

- ol_material

Each order specifies the material to be used. We can join to the material table to find a description or cost per meter. Material 1 is Silk, Black, Plain costing £7 per meter.


In [3]:
order_line.head()

Row(order_ref=1, line_no=1, ol_style=1, ol_size=8, ol_material=1)

## 2.
A sample join:

In order to translate the numbers in order_line into meaningful values we need to join a related table. For example if we want to access the descriptions of the materials we need to join the material table.

To achieve the join we include the table material on the FROM line and the join condition as a WHERE clause.

For each pair of tables there is a join condition between them (if they are linked). To find the join condition between order_line and material we look at the order_line table CREATE statement and notice the line that specifies that ol_material references the material table. This link will always be to the primary key of material table.

```sql
CREATE TABLE order_line (
  order_ref	INTEGER	NOT NULL REFERENCES dress_order
 ,line_no	INTEGER	NOT NULL
 ,ol_style	INTEGER	REFERENCES garment
 ,ol_size	INTEGER	NOT NULL
 ,ol_material	INTEGER	REFERENCES material
 ,PRIMARY KEY (order_ref, line_no)
 ,FOREIGN KEY (ol_style, ol_size) REFERENCES quantities
 );
SELECT order_ref, line_no, fabric, colour, pattern, cost
  FROM order_line, material
 WHERE ol_material = material_no
```

In [4]:
(order_line
 .join(material, on=(order_line['ol_material']==material['material_no']))
 .select('order_ref', 'line_no', 'fabric', 'colour', 'pattern', 'cost')
 .toPandas())

,order_ref,line_no,fabric,colour,pattern,cost
0,12,3,Silk,Black,Plain,7.00
1,7,1,Silk,Black,Plain,7.00
2,1,1,Silk,Black,Plain,7.00
3,12,4,Silk,Red Abstract,Printed,10.00
4,7,2,Silk,Red Abstract,Printed,10.00
5,1,2,Silk,Red Abstract,Printed,10.00
6,12,5,Cotton,Yellow Stripe,Woven,3.00
7,7,3,Cotton,Yellow Stripe,Woven,3.00
8,2,1,Cotton,Yellow Stripe,Woven,3.00
9,8,1,Cotton,Green Stripe,Woven,3.00


## 3.
To get a description of the garment we need to join the garment table. The join condition is that the ol_style in order_line matches the style_no in garment.

```sql
SELECT order_ref, line_no, description
  FROM order_line, garment
 WHERE ol_style = style_no
```

In [5]:
(order_line
 .join(garment, on=(order_line['ol_style']==garment['style_no']))
 .select('order_ref', 'line_no', 'description')
 .toPandas())

,order_ref,line_no,description
0,1,1,Trousers
1,1,2,Long Skirt
2,2,1,Shorts
3,2,2,Short Skirt
4,2,3,Sundress
5,3,1,Suntop
6,4,1,Trousers
7,4,2,Long Skirt
8,5,1,Shorts
9,5,2,Short Skirt


## 4.
If we need both the description and the fabric we can join both material and garment to the order_line table. The join conditions are combined with "AND"

```sql
SELECT order_ref, line_no, fabric, description
  FROM order_line, material, garment
 WHERE ol_style = style_no
   AND ol_material = material_no
```

In [6]:
(order_line
 .join(material, on=(order_line['ol_material']==material['material_no']))
 .join(garment, on=(order_line['ol_style']==garment['style_no']))
 .select('order_ref', 'line_no', 'fabric', 'description')
 .toPandas())

,order_ref,line_no,fabric,description
0,12,3,Silk,Suntop
1,7,1,Silk,Short Skirt
2,1,1,Silk,Trousers
3,12,4,Silk,Trousers
4,7,2,Silk,Sundress
5,1,2,Silk,Long Skirt
6,12,5,Cotton,Long Skirt
7,7,3,Cotton,Suntop
8,2,1,Cotton,Shorts
9,8,1,Cotton,Suntop


## 5.
The quantities table tells us how much material is required for every garment for every size available. The join between the order_line and quantities is unusual in that it involves two fields. This can be seen by the fact that quantities has a composite key.

```sql
SELECT order_ref, line_no, quantity
  FROM order_line, quantities
 WHERE ol_style = style_q
   AND ol_size  = size_q
```

In [7]:
(order_line
 .join(quantities, on=((order_line['ol_style']==quantities['style_q']) & 
                       (order_line['ol_size']==quantities['size_q'])))
 .select('order_ref', 'line_no', 'quantity')
 .toPandas())

,order_ref,line_no,quantity
0,12,4,2.7
1,6,1,2.7
2,1,1,2.7
3,4,1,2.8
4,10,2,3.0
5,11,1,3.0
6,12,5,3.4
7,6,2,3.4
8,1,2,3.4
9,4,2,3.8


## 6.
Customers place orders - each order contains many lines - each line of the order refers to a garment:

```sql
SELECT c_name, order_date, order_no, line_no
   FROM jmcust, dress_order, order_line
  WHERE          jmcust.c_no = dress_order.cust_no
  AND   dress_order.order_no = order_line.order_ref
```

In [8]:
(jmcust.join(dress_order, on=(jmcust['c_no']==dress_order['cust_no']))
 .join(order_line, on=(dress_order['order_no']==order_line['order_ref']))
 .select('c_name', 'order_date', 'order_no', 'line_no')
 .toPandas())

,c_name,order_date,order_no,line_no
0,Ms Black,2002-02-27,8,3
1,Ms Black,2002-02-27,8,2
2,Ms Black,2002-02-27,8,1
3,Ms Brown,2002-02-27,9,1
4,Ms Brown,2002-02-21,7,3
5,Ms Brown,2002-02-21,7,2
6,Ms Brown,2002-02-21,7,1
7,Ms Gray,2002-02-28,10,2
8,Ms Gray,2002-02-28,10,1
9,Ms Gray,2002-02-20,6,3


## 7.
There's also a dress maker table, and a table called construction which gives you information about who made which order and when:

```sql
SELECT d_no, d_name, construction.order_ref, construction.line_ref, start_date, finish_date
 FROM dressmaker, order_line, construction
 WHERE  d_no=maker  
 AND order_line.order_ref=construction.order_ref 
 AND order_line.line_no=construction.line_ref
```

In [9]:
(dressmaker
 .join(construction, on=(dressmaker['d_no']==construction['maker']))
 .join(order_line, on=((construction['order_ref']==order_line['order_ref']) &
                       (construction['line_ref']==order_line['line_no'])))
 .select('d_no', 'd_name', col('order_line.order_ref'), 'line_ref', 
         'start_date', 'finish_date')
 .toPandas())

,d_no,d_name,order_ref,line_ref,start_date,finish_date
0,1,Mrs Hem,1,1,2002-01-10,2002-03-05
1,1,Mrs Hem,4,2,2002-02-02,2002-03-25
2,1,Mrs Hem,7,1,2002-02-21,None
3,1,Mrs Hem,10,1,2002-02-28,None
4,1,Mrs Hem,12,3,2002-03-03,None
5,2,Miss Stitch,1,2,2002-01-10,2002-03-15
6,2,Miss Stitch,5,1,2002-02-03,2002-03-15
7,2,Miss Stitch,7,2,2002-02-21,None
8,2,Miss Stitch,10,2,2002-03-28,None
9,2,Miss Stitch,12,4,2002-03-03,None


In [10]:
sc.stop()